In [1]:
import pandas as pd
from services.service_sptrans import ServiceSPTRANS
import folium
from time import sleep
from entidades.posicao import Posicao
from entidades.trajeto import Trajeto

In [32]:
ss = ServiceSPTRANS()
linhas = ss.consultar_linha('701U-10')
linhas

In [33]:
linhas[0].trajeto.posicoes

 ...]

# Mapa dos pontos de ônibus

In [46]:
def atualizar_posicao_onibus(linhas):
    while True:
        mapa_parada = folium.Map(location=[-23.5489, -46.6388], zoom_start=10)

        for linha in linhas:
            lista_posicoes_veiculos = ss.buscar_posicoes_veiculos(linha.codigo_identificador)

            for i in range(len(linha.trajeto.posicoes) - 1):
                ponto_inicial = [linha.trajeto.posicoes[i].latitude, linha.trajeto.posicoes[i].longitude]
                ponto_final = [linha.trajeto.posicoes[i + 1].latitude, linha.trajeto.posicoes[i + 1].longitude]
                folium.PolyLine(locations=[ponto_inicial, ponto_final], color=f'#{linha.trajeto.cor}').add_to(
                    mapa_parada)

            for posicao_veiculo in lista_posicoes_veiculos:
                folium.Marker([posicao_veiculo.posicao.latitude, posicao_veiculo.posicao.longitude],
                              popup=posicao_veiculo.prefixo, icon=folium.Icon(icon='bus', prefix='fa', color='green')).add_to(mapa_parada)

        paradas = ss.buscar_paradas(linha.codigo_identificador)
        for parada in paradas:
            folium.Marker([parada.posicao.latitude, parada.posicao.longitude], popup=parada.nome_parada , icon=folium.Icon(icon='bus', prefix='fa', color='blue')).add_to(
                mapa_parada)

        mapa_parada.save('parada6.html')
        print('Atualizei')
        sleep(10)


In [47]:
atualizar_posicao_onibus(linhas)

Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei
Atualizei


KeyboardInterrupt: 

# Teste para obter a cor do trajeto da linha e as posições

In [29]:
base_trips = pd.read_csv('../data/raw/trips.txt', usecols=['route_id', 'shape_id'])
base_shapes = pd.read_csv('../data/raw/shapes.txt',
                          usecols=['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_dist_traveled'])
base_cor_trajeto = pd.read_csv('../data/raw/routes.txt', usecols=['route_id', 'route_color'])

base_completa = base_trips.merge(base_shapes, on='shape_id', how='inner')

,route_id,shape_id,shape_pt_lat,shape_pt_lon,shape_dist_traveled
0,1012-10,72757,-23.432174,-46.787095,0.000000
1,1012-10,72757,-23.432116,-46.787105,6.504682
2,1012-10,72757,-23.431590,-46.787191,65.418427
3,1012-10,72757,-23.431476,-46.787206,78.136925
4,1012-10,72757,-23.431438,-46.787210,82.364815
...,...,...,...,...,...
1125639,N843-11,62557,-23.654782,-46.761548,24200.133000
1125640,N843-11,62557,-23.654284,-46.762232,24289.080000
1125641,N843-11,62557,-23.654177,-46.762379,24308.193000
1125642,N843-11,62557,-23.654077,-46.762517,24326.105000


In [35]:
base_cor_trajeto[base_cor_trajeto['route_id'] == '8000-10']['route_color'].iloc[0]

'509E2F'

In [20]:
base_filtrada = base_completa.loc[base_completa['route_id'] == '8000-10', ['shape_pt_lat', 'shape_pt_lon']]


In [50]:
lista_posicoes = base_filtrada.values.tolist()
lista_posicoes

[[-23.520438000000002, -46.699642],
 [-23.520460999999997, -46.699581],
 [-23.520559, -46.699323],
 [-23.520663, -46.699366999999995],
 [-23.521096, -46.699542],
 [-23.521053, -46.699669],
 [-23.520812, -46.700337],
 [-23.521482000000002, -46.700390999999996],
 [-23.521648000000003, -46.700406],
 [-23.521703, -46.700408],
 [-23.522289, -46.70044],
 [-23.522759, -46.700465],
 [-23.523410000000002, -46.700502],
 [-23.523801000000002, -46.700524],
 [-23.524862, -46.700585],
 [-23.524916, -46.699237],
 [-23.524941000000002, -46.697779],
 [-23.52497, -46.696641],
 [-23.524957999999998, -46.695528],
 [-23.524966, -46.694325],
 [-23.524971, -46.693151],
 [-23.52497, -46.692291],
 [-23.524975, -46.690844],
 [-23.524979000000002, -46.689758000000005],
 [-23.524983, -46.688499],
 [-23.525005, -46.687494],
 [-23.525007000000002, -46.687239],
 [-23.525011, -46.686823],
 [-23.525015, -46.686153999999995],
 [-23.525014000000002, -46.685392],
 [-23.525023, -46.684560999999995],
 [-23.525028, -46.684]

In [47]:
posicoes = []
for posicao in lista_posicoes:
    p = Posicao(posicao[0], posicao[1])
    posicoes.append(p)

In [51]:
posicoes = base_filtrada.apply(lambda row: Posicao(row['shape_pt_lat'], row['shape_pt_lon']), axis=1).tolist()

In [97]:
posicoes = base_filtrada.apply(lambda row: Posicao(row['shape_pt_lat'], row['shape_pt_lon']), axis=1).tolist()

lista_de_lista = []
for posicao in posicoes:
    lista_de_lista.append([posicao])

In [104]:
posicoes

In [140]:
linha = '8000-10'
t = Trajeto(linha)

mapa_trajeto_teste = folium.Map(location=[-23.5489, -46.6388], zoom_start=10)

for i in range(len(t.posicoes) - 1):
    ponto_inicial = [t.posicoes[i].latitude, t.posicoes[i].longitude]
    ponto_final = [t.posicoes[i + 1].latitude, t.posicoes[i + 1].longitude]
    folium.PolyLine(locations=[ponto_inicial, ponto_final], color=f'#{t.cor}').add_to(mapa_trajeto_teste)

mapa_trajeto_teste.save('mapa_trajeto_teste.html')


In [126]:
for i in range(len(t.posicoes) - 1):
    ponto_inical = t.posicoes[i]
    ponto_final = t.posicoes[i + 1]

    print('ponto_inical', i, ponto_inical.latitude, ponto_final.longitude)
    print('ponto_final', i, ponto_final.latitude, ponto_final.longitude)

ponto_inical 0 -23.520438000000002 -46.699581
ponto_final 0 -23.520460999999997 -46.699581
ponto_inical 1 -23.520460999999997 -46.699323
ponto_final 1 -23.520559 -46.699323
ponto_inical 2 -23.520559 -46.699366999999995
ponto_final 2 -23.520663 -46.699366999999995
ponto_inical 3 -23.520663 -46.699542
ponto_final 3 -23.521096 -46.699542
ponto_inical 4 -23.521096 -46.699669
ponto_final 4 -23.521053 -46.699669
ponto_inical 5 -23.521053 -46.700337
ponto_final 5 -23.520812 -46.700337
ponto_inical 6 -23.520812 -46.700390999999996
ponto_final 6 -23.521482000000002 -46.700390999999996
ponto_inical 7 -23.521482000000002 -46.700406
ponto_final 7 -23.521648000000003 -46.700406
ponto_inical 8 -23.521648000000003 -46.700408
ponto_final 8 -23.521703 -46.700408
ponto_inical 9 -23.521703 -46.70044
ponto_final 9 -23.522289 -46.70044
ponto_inical 10 -23.522289 -46.700465
ponto_final 10 -23.522759 -46.700465
ponto_inical 11 -23.522759 -46.700502
ponto_final 11 -23.523410000000002 -46.700502
ponto_inical 1

In [132]:
coordenadas = [[-23.520438000000002, -46.699581],
               [-23.520460999999997, -46.699323],
               [-23.520559, -46.699366999999995],
               [-23.520663, -46.699542],
               [-23.521096, -46.699669],
               ]
mapa_trajeto_teste = folium.Map(location=[-23.5489, -46.6388], zoom_start=17, control_scale=True)

for i in range(len(coordenadas) - 1):
    ponto_inical = coordenadas[i]
    ponto_final = coordenadas[i - 1]
    print(i - 1)
    print('ponto_inical', ponto_inical)
    print('ponto_final', ponto_final)
    folium.PolyLine(locations=[ponto_inical, ponto_final]).add_to(mapa_trajeto_teste)
mapa_trajeto_teste.save('teste.html')

-1
ponto_inical [-23.520438000000002, -46.699581]
ponto_final [-23.521096, -46.699669]
0
ponto_inical [-23.520460999999997, -46.699323]
ponto_final [-23.520438000000002, -46.699581]
1
ponto_inical [-23.520559, -46.699366999999995]
ponto_final [-23.520460999999997, -46.699323]
2
ponto_inical [-23.520663, -46.699542]
ponto_final [-23.520559, -46.699366999999995]
